In [10]:
import pandas as pd
import math

# LEITURA DE DADOS

# Leitura da planilha Dados_cliente1 (FATURA)
df_fatura = pd.read_excel('../data/raw/Dados_cliente.xlsx')

# Leitura da planilha Dados_cliente1 (INFORMACOES)
df_informacoes = pd.read_excel('../data/raw/Dados_cliente.xlsx', sheet_name=1)

# Leitura da planilha HSP-RS
df_hsp = pd.read_excel('../data/raw/HSP-RS.xlsx')

display(df_fatura.describe())

,MÊS/ANO,CONSUMO (KWH)
count,12,12.000000
mean,2025-06-16 12:00:00,370.083333
min,2025-01-01 00:00:00,228.000000
25%,2025-03-24 06:00:00,318.500000
50%,2025-06-16 00:00:00,373.000000
75%,2025-09-08 12:00:00,415.250000
max,2025-12-01 00:00:00,574.000000
std,NaN,91.479216


In [11]:
# CALCULAR MÉDIA APARADA COM BASE EM UMA NOVA COLUNA

# Cria nova coluna para remover os valores máximo e mínimo
df_fatura['CONSUMO (KWH) APARADO'] = df_fatura['CONSUMO (KWH)']

# Valor max = 0
max_index = df_fatura['CONSUMO (KWH) APARADO'].idxmin()
df_fatura.loc[max_index, 'CONSUMO (KWH) APARADO'] = 0

#Valor min = 0
min_index = df_fatura['CONSUMO (KWH) APARADO'].idxmax()
df_fatura.loc[min_index, 'CONSUMO (KWH) APARADO'] = 0

display(df_fatura)

# Calcula média aparada desconsiderando linhas com consumo = 0 
consumo_medio = df_fatura[df_fatura['CONSUMO (KWH) APARADO'] != 0] ['CONSUMO (KWH) APARADO'].mean()
print('Consumo médio mensal:',consumo_medio,'kWh')

,MÊS/ANO,CONSUMO (KWH),CONSUMO (KWH) APARADO
0,2025-01-01,574,0
1,2025-02-01,410,410
2,2025-03-01,372,372
3,2025-04-01,431,431
4,2025-05-01,374,374
5,2025-06-01,352,352
6,2025-07-01,319,319
7,2025-08-01,228,0
8,2025-09-01,247,247
9,2025-10-01,385,385


Consumo médio mensal: 363.9 kWh


In [12]:
# CALCULAR CAPACIDADE DO SISTEMA

# Calcula media de consumo diária 
consumo_medio_diario = consumo_medio / 30

# Define o valor de horas de sol pleno comparando a lista de horas do RS com a cidade do cliente
hsp = df_hsp.loc[df_hsp['CIDADE'].isin(df_informacoes['CIDADE']), 'HSP'].item()

# Fórmula para calcular a capacidade do sistema (20% de perda)
capacidade_sistema_kwp = consumo_medio_diario / (hsp * 0.8)

display(df_informacoes)
display(df_hsp)
print('Horas de sol pleno:',hsp)
print(f'Capacidade do sistema: {capacidade_sistema_kwp:.2f} kWp')

,CIDADE,PAINEL (W)
0,Porto Alegre,550


,CIDADE,HSP
0,Porto Alegre,4.42
1,Caxias do Sul,4.35
2,Canoas,4.40
3,Pelotas,4.38
4,Santa Maria,4.45
5,Gravataí,4.39
6,Novo Hamburgo,4.37
7,Viamão,4.41
8,São Leopoldo,4.38
9,Passo Fundo,4.48


Horas de sol pleno: 4.42
Capacidade do sistema: 3.43 kWp


In [13]:
# CALCULAR A QUANTIDADE DE MÓDULOS

# Converter kWp para Wp
capacidade_sistema_wp = capacidade_sistema_kwp * 1000

# Calcular a quantidade de módulos 
qtd_modulos = math.ceil(capacidade_sistema_wp / (df_informacoes['PAINEL (W)'].item()))

print('Quantidade de módulos fotovoltaicos:',qtd_modulos)

Quantidade de módulos fotovoltaicos: 7
